In [1]:
# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from PyPDF2 import PdfFileReader

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [2]:
raw_file = 'JPM-2017-MachineLearningInvestments.pdf'
open_file = open(raw_file, 'rb')
pdf = PdfFileReader(open_file)

num_pages = pdf.getNumPages()

In [3]:
pdf_text = ''
for pages in range(num_pages):
    page = pdf.getPage(pages)
    pdf_text = pdf_text + ' ' + page.extractText()

In [4]:
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(pdf_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\evere\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

Token indices sequence length is longer than the specified maximum sequence length for this model (2330 > 1024). Running this sequence through the model will result in indexing errors


2330

In [6]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

237

In [11]:
[len(tokenizer.tokenize(c)) for c in chunks]

[1010,
 879,
 997,
 990,
 981,
 970,
 1012,
 972,
 991,
 1021,
 944,
 994,
 987,
 938,
 1001,
 1013,
 1012,
 999,
 975,
 1017,
 995,
 1015,
 1017,
 947,
 1017,
 1014,
 983,
 928,
 1010,
 998,
 890,
 993,
 1013,
 912,
 980,
 996,
 1001,
 1013,
 943,
 1003,
 998,
 956,
 1001,
 1003,
 1014,
 1008,
 753,
 999,
 1007,
 927,
 985,
 1002,
 874,
 930,
 987,
 929,
 935,
 747,
 2330,
 943,
 986,
 797,
 981,
 958,
 868,
 1003,
 975,
 1017,
 955,
 991,
 992,
 1015,
 991,
 956,
 950,
 1008,
 1014,
 1008,
 1001,
 968,
 930,
 894,
 914,
 835,
 1008,
 1002,
 999,
 513,
 993,
 1007,
 982,
 862,
 992,
 994,
 885,
 893,
 971,
 921,
 1015,
 992,
 859,
 527,
 1018,
 832,
 1014,
 935,
 908,
 1011,
 938,
 1013,
 1015,
 950,
 1000,
 1003,
 973,
 1009,
 990,
 1009,
 968,
 1001,
 1002,
 985,
 968,
 1011,
 1002,
 994,
 998,
 967,
 902,
 905,
 1004,
 1007,
 988,
 976,
 998,
 1010,
 1001,
 1019,
 982,
 1020,
 1003,
 978,
 1012,
 978,
 984,
 944,
 986,
 975,
 1016,
 970,
 896,
 1003,
 1017,
 988,
 1000,
 993,
 962,

In [8]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [12]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

 Big Data and AI Strategies Machine Learning and Alternative Data Approach to Investing Quantitative and Derivatives Strategy. This document is being provided for the exclusive use of LOGAN SCOTT at JPMorgan Chase & Co. and clients of J.P. Morgan. The report provides a framework for the use of machine learning and artificial intelligence in investing.
 This guide is being provided for the exclusive use of LOGAN SCOTT at JPMorgan Chase & Co. and clients of J.P. Morgan. We hope this guide will be educative for investors new to the concept of Big Data and Machine Learning, and provide new insights and perspectives to those who already practice it.
 This document is being provided for the exclusive use of LOGAN SCOTT at JPMorgan Chase & Co. and clients of J.P. Morgan. We aim to provide a roadmap to different types of data and methods to assess different asset classes for different investment styles for their relevance.
 Machine Learning and Big Data will become a standard tool for quantita

KeyboardInterrupt: 